# How-to

This notebook serves as a practical guide to common questions users might have.

In [ ]:
import MDAnalysis as mda
import prolif as plf

## Changing the parameters for an interaction

In [ ]:
# TODO

## Writing your own interaction

In [ ]:
# TODO

## Ignoring the protein backbone when computing interactions

In [ ]:
# TODO